In [1]:
# Import our dependencies
import numpy as np
import pandas as pd
import polars as pl

In [2]:
#  Import and read the input csv
NMp10k_df = pd.read_csv('../../Raw_Data/WHO_Nurses_midwifery_per10k-data.csv')
NMp10k_df.head(5)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,HWF_0006,"Nursing and midwifery personnel (per 10,000)",numeric,AFR,Africa,Country,TCD,Chad,Year,2023,...,NaN,NaN,NaN,NaN,NaN,1.67,NaN,"NHWA data portal, December 2024 update https:/...",EN,2025-01-14T05:00:00.000Z
1,HWF_0006,"Nursing and midwifery personnel (per 10,000)",numeric,WPR,Western Pacific,Country,LAO,Lao People's Democratic Republic,Year,2023,...,NaN,NaN,NaN,NaN,NaN,10.27,NaN,"NHWA data portal, December 2024 update https:/...",EN,2025-01-14T05:00:00.000Z
2,HWF_0006,"Nursing and midwifery personnel (per 10,000)",numeric,AFR,Africa,Country,CAF,Central African Republic,Year,2023,...,NaN,NaN,NaN,NaN,NaN,10.97,NaN,"NHWA data portal, December 2024 update https:/...",EN,2025-01-14T05:00:00.000Z
3,HWF_0006,"Nursing and midwifery personnel (per 10,000)",numeric,EUR,Europe,Country,BLR,Belarus,Year,2023,...,NaN,NaN,NaN,NaN,NaN,108.15,NaN,"NHWA data portal, December 2024 update https:/...",EN,2025-01-14T05:00:00.000Z
4,HWF_0006,"Nursing and midwifery personnel (per 10,000)",numeric,AMR,Americas,Country,CAN,Canada,Year,2023,...,NaN,NaN,NaN,NaN,NaN,112.57,NaN,"NHWA data portal, December 2024 update https:/...",EN,2025-01-14T05:00:00.000Z


In [3]:
# Drop the population estimate column
NMp10k_df = NMp10k_df.drop(['IndicatorCode','ValueType','ParentLocationCode','ParentLocation','Location type','SpatialDimValueCode','Period type'], axis=1)
NMp10k_df = NMp10k_df.drop(['FactValueUoM','FactValueNumericLowPrefix','FactValueNumericLow','FactValueNumericHighPrefix','FactValueNumericHigh'], axis=1)
NMp10k_df = NMp10k_df.drop(['FactValueTranslationID','FactComments','Language','DateModified','IsLatestYear','Dim1 type','Dim1','Dim2','Dim3',], axis=1)
NMp10k_df = NMp10k_df.drop(['Dim1ValueCode','Dim2ValueCode','Dim3ValueCode','Dim2 type','Dim3 type','DataSource','FactValueNumericPrefix'], axis=1)
NMp10k_df = NMp10k_df.drop(['FactValueNumeric','DataSourceDimValueCode','Indicator'], axis=1)
NMp10k_df.rename(columns ={'Location':'Country', 'Period':'Year', 'Value':'RN/Midwife per 10k'}, inplace = True)
NMp10k_df

,Country,Year,RN/Midwife per 10k
0,Chad,2023,1.67
1,Lao People's Democratic Republic,2023,10.27
2,Central African Republic,2023,10.97
3,Belarus,2023,108.15
4,Canada,2023,112.57
...,...,...,...
3409,Turkmenistan,1990,82.17
3410,Estonia,1990,87.95
3411,Brazil,1990,9.49
3412,New Zealand,1990,92.01


In [4]:
# Open the list of country name corrections
corrections_df = pd.read_csv('../../Clean_Data/master_country_list/country_name_corrections.csv')

# Convert the corrections dataframe to a dictionary.
correction_dict = dict(zip(corrections_df['wrong'], corrections_df['correct']))

In [5]:
# Apply the correction dictionary to fix the known errors
NMp10k_df['Country'] = NMp10k_df['Country'].replace(correction_dict)

In [6]:
# Open the master list of countries
countries_df = pd.read_csv('../../Clean_Data/master_country_list/country_profile_urls.csv')
countries_df = countries_df.drop(['profile_url'], axis=1)

In [7]:
country_list = countries_df['country'].tolist()
#country_list

In [8]:
# Create a list of MDp10k countries
NMp10k_countries = NMp10k_df['Country'].tolist()
#NMp10k_countries

In [9]:
# clean the FS data based on the SS country list
no_match = []

for country in NMp10k_countries:
    if country in country_list:
        continue
    else:
        no_match.append(country)

In [10]:
NMp10k_clean_df = NMp10k_df.drop(NMp10k_df[NMp10k_df['Country'].isin(no_match)].index.tolist())
NMp10k_clean_df

,Country,Year,RN/Midwife per 10k
0,Chad,2023,1.67
1,Laos,2023,10.27
2,Central African Republic,2023,10.97
3,Belarus,2023,108.15
4,Canada,2023,112.57
...,...,...,...
3409,Turkmenistan,1990,82.17
3410,Estonia,1990,87.95
3411,Brazil,1990,9.49
3412,New Zealand,1990,92.01


In [11]:
NMp10k_sorted_df = NMp10k_clean_df.sort_values(['Country','Year'], axis=0)

In [12]:
NMp10k_sorted_df.tail(40)

,Country,Year,RN/Midwife per 10k
1086,Vietnam,2015,14.210
943,Vietnam,2016,14.410
3043,Yemen,1997,3.710
2490,Yemen,2004,6.220
1807,Yemen,2010,6.200
1469,Yemen,2013,7.420
1333,Yemen,2014,7.510
1159,Yemen,2015,6.160
1023,Yemen,2016,6.330
874,Yemen,2017,6.460


In [13]:
# Create a complete dataframe with all countries and all the years.

# Define variables to collect the details.
#data_countries = MDp10k_sorted_df['Country'].unique()
data_countries = country_list
years = list(range(1990, 2024))

# Create a helper DataFrame.
helper_df = pd.DataFrame({'Country':np.repeat(data_countries, len(years)), 'Year':np.tile(years, len(data_countries))})

# Merge the helper DataFrame with the original data to ensure the data range is complete.
complete_df = pd.merge(helper_df, NMp10k_sorted_df, on=['Country','Year'], how='left')

In [14]:
# Convert to wide format
wide_df = complete_df.pivot(index='Year', columns='Country', values='RN/Midwife per 10k')

# Flatten the hierarchical columns and create 'Country_Series' style column names
wide_df.columns = [f'{col}_RN/Midwife per 10k' for col in wide_df.columns]

# Reset the index, so 'Year' becomes a column again
wide_df.reset_index(inplace=True)

# Convert columns to numeric, coerce non-numeric values to NaN
for col in wide_df.columns[0:]:
    wide_df[col] = pd.to_numeric(wide_df[col], errors='coerce')

# Convert to Polars DataFrame
wide_pl_df = pl.from_pandas(wide_df)
    
wide_pl_df.head(5)

Year,Albania_RN/Midwife per 10k,Algeria_RN/Midwife per 10k,Andorra_RN/Midwife per 10k,Angola_RN/Midwife per 10k,Antigua and Barbuda_RN/Midwife per 10k,Argentina_RN/Midwife per 10k,Armenia_RN/Midwife per 10k,Aruba_RN/Midwife per 10k,Australia_RN/Midwife per 10k,Austria_RN/Midwife per 10k,Azerbaijan_RN/Midwife per 10k,Bahamas_RN/Midwife per 10k,Bahrain_RN/Midwife per 10k,Bangladesh_RN/Midwife per 10k,Barbados_RN/Midwife per 10k,Belarus_RN/Midwife per 10k,Belgium_RN/Midwife per 10k,Belize_RN/Midwife per 10k,Benin_RN/Midwife per 10k,Bermuda_RN/Midwife per 10k,Bhutan_RN/Midwife per 10k,Bolivia_RN/Midwife per 10k,Bosnia and Herzegovina_RN/Midwife per 10k,Botswana_RN/Midwife per 10k,Brazil_RN/Midwife per 10k,British Virgin Islands_RN/Midwife per 10k,Brunei_RN/Midwife per 10k,Bulgaria_RN/Midwife per 10k,Burkina Faso_RN/Midwife per 10k,Burundi_RN/Midwife per 10k,Cabo Verde_RN/Midwife per 10k,Cambodia_RN/Midwife per 10k,Cameroon_RN/Midwife per 10k,Canada_RN/Midwife per 10k,Central African Republic_RN/Midwife per 10k,Chad_RN/Midwife per 10k,…,Serbia_RN/Midwife per 10k,Seychelles_RN/Midwife per 10k,Sierra Leone_RN/Midwife per 10k,Singapore_RN/Midwife per 10k,Slovakia_RN/Midwife per 10k,Slovenia_RN/Midwife per 10k,Solomon Islands_RN/Midwife per 10k,South Africa_RN/Midwife per 10k,South Korea_RN/Midwife per 10k,Spain_RN/Midwife per 10k,Sri Lanka_RN/Midwife per 10k,Sudan_RN/Midwife per 10k,Suriname_RN/Midwife per 10k,Sweden_RN/Midwife per 10k,Switzerland_RN/Midwife per 10k,Syria_RN/Midwife per 10k,Taiwan (China)_RN/Midwife per 10k,Tajikistan_RN/Midwife per 10k,Tanzania_RN/Midwife per 10k,Thailand_RN/Midwife per 10k,Togo_RN/Midwife per 10k,Trinidad and Tobago_RN/Midwife per 10k,Tunisia_RN/Midwife per 10k,Turkmenistan_RN/Midwife per 10k,Türkiye_RN/Midwife per 10k,Uganda_RN/Midwife per 10k,Ukraine_RN/Midwife per 10k,United Kingdom_RN/Midwife per 10k,United States_RN/Midwife per 10k,Uruguay_RN/Midwife per 10k,Uzbekistan_RN/Midwife per 10k,Vanuatu_RN/Midwife per 10k,Venezuela_RN/Midwife per 10k,Vietnam_RN/Midwife per 10k,Yemen_RN/Midwife per 10k,Zambia_RN/Midwife per 10k,Zimbabwe_RN/Midwife per 10k
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1990,45.1,null,null,null,null,null,75.07,null,115.9,40.48,111.95,null,null,null,null,74.96,null,null,null,null,null,null,50.62,null,9.49,null,null,69.54,null,null,null,null,null,110.68,null,null,…,null,null,null,null,null,53.89,null,null,null,28.3,7.81,null,null,133.15,null,null,null,58.63,null,null,null,null,null,82.17,null,null,null,null,103.94,null,114.29,null,null,null,null,null,14.96
1991,null,null,null,null,null,null,74.52,null,120.32,41.65,113.5,null,null,null,null,71.26,null,null,null,null,null,null,49.21,null,null,null,null,66.39,null,null,null,null,null,111.86,null,null,…,null,null,null,null,null,60.0,null,null,null,41.14,8.53,null,null,136.68,null,null,null,60.61,null,7.31,null,null,null,84.88,null,null,null,null,106.76,null,118.36,null,null,null,null,null,null
1992,null,null,null,null,null,20.48,73.86,null,115.28,43.44,109.61,null,null,null,null,73.4,null,null,null,null,null,null,null,null,11.45,null,null,66.89,null,null,null,null,null,111.48,null,null,…,null,null,null,null,null,60.11,null,null,null,41.83,9.2,null,null,140.75,null,null,null,57.46,null,null,null,null,null,85.99,null,null,null,null,107.42,null,118.57,null,null,null,null,null,null
1993,48.24,null,null,null,null,null,74.89,null,110.44,45.94,110.24,null,null,null,null,76.73,null,null,null,null,null,null,null,null,12.65,null,null,69.62,null,null,null,null,null,110.45,null,null,…,null,null,null,null,null,58.97,null,null,null,42.34,10.84,null,null,144.53,null,null,null,56.54,null,8.13,null,null,null,86.46,15.32,null,null,null,108.99,null,118.15,null,null,null,null,null,null
1994,49.58,null

In [15]:
# Handle the missing data

wide_pl_df = wide_pl_df.interpolate()
wide_pl_df = wide_pl_df.fill_null(strategy='backward')
wide_pl_df = wide_pl_df.fill_null(strategy='forward')
wide_pl_df = wide_pl_df.fill_null(strategy='zero')

In [16]:
wide_pl_df.head(15)

Year,Albania_RN/Midwife per 10k,Algeria_RN/Midwife per 10k,Andorra_RN/Midwife per 10k,Angola_RN/Midwife per 10k,Antigua and Barbuda_RN/Midwife per 10k,Argentina_RN/Midwife per 10k,Armenia_RN/Midwife per 10k,Aruba_RN/Midwife per 10k,Australia_RN/Midwife per 10k,Austria_RN/Midwife per 10k,Azerbaijan_RN/Midwife per 10k,Bahamas_RN/Midwife per 10k,Bahrain_RN/Midwife per 10k,Bangladesh_RN/Midwife per 10k,Barbados_RN/Midwife per 10k,Belarus_RN/Midwife per 10k,Belgium_RN/Midwife per 10k,Belize_RN/Midwife per 10k,Benin_RN/Midwife per 10k,Bermuda_RN/Midwife per 10k,Bhutan_RN/Midwife per 10k,Bolivia_RN/Midwife per 10k,Bosnia and Herzegovina_RN/Midwife per 10k,Botswana_RN/Midwife per 10k,Brazil_RN/Midwife per 10k,British Virgin Islands_RN/Midwife per 10k,Brunei_RN/Midwife per 10k,Bulgaria_RN/Midwife per 10k,Burkina Faso_RN/Midwife per 10k,Burundi_RN/Midwife per 10k,Cabo Verde_RN/Midwife per 10k,Cambodia_RN/Midwife per 10k,Cameroon_RN/Midwife per 10k,Canada_RN/Midwife per 10k,Central African Republic_RN/Midwife per 10k,Chad_RN/Midwife per 10k,…,Serbia_RN/Midwife per 10k,Seychelles_RN/Midwife per 10k,Sierra Leone_RN/Midwife per 10k,Singapore_RN/Midwife per 10k,Slovakia_RN/Midwife per 10k,Slovenia_RN/Midwife per 10k,Solomon Islands_RN/Midwife per 10k,South Africa_RN/Midwife per 10k,South Korea_RN/Midwife per 10k,Spain_RN/Midwife per 10k,Sri Lanka_RN/Midwife per 10k,Sudan_RN/Midwife per 10k,Suriname_RN/Midwife per 10k,Sweden_RN/Midwife per 10k,Switzerland_RN/Midwife per 10k,Syria_RN/Midwife per 10k,Taiwan (China)_RN/Midwife per 10k,Tajikistan_RN/Midwife per 10k,Tanzania_RN/Midwife per 10k,Thailand_RN/Midwife per 10k,Togo_RN/Midwife per 10k,Trinidad and Tobago_RN/Midwife per 10k,Tunisia_RN/Midwife per 10k,Turkmenistan_RN/Midwife per 10k,Türkiye_RN/Midwife per 10k,Uganda_RN/Midwife per 10k,Ukraine_RN/Midwife per 10k,United Kingdom_RN/Midwife per 10k,United States_RN/Midwife per 10k,Uruguay_RN/Midwife per 10k,Uzbekistan_RN/Midwife per 10k,Vanuatu_RN/Midwife per 10k,Venezuela_RN/Midwife per 10k,Vietnam_RN/Midwife per 10k,Yemen_RN/Midwife per 10k,Zambia_RN/Midwife per 10k,Zimbabwe_RN/Midwife per 10k
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1990.0,45.1,21.97,24.57,9.3,31.62,20.48,75.07,0.0,115.9,40.48,111.95,41.95,29.59,1.38,51.91,74.96,92.33,12.58,7.1,0.0,7.92,20.67,50.62,26.54,9.49,81.633,39.71,69.54,4.86,6.37,9.8,10.77,15.68,110.68,3.85,0.47,…,58.34,71.88,2.26,40.2,74.17,53.89,8.39,7.08,10.93,28.3,7.81,10.31,14.44,133.15,118.74,21.9,0.0,58.63,3.68,7.31,3.4,28.85,20.53,82.17,15.32,6.99,65.0,85.77,103.94,8.78,114.29,24.51,6.49,7.59,3.71,7.14,14.96
1991.0,46.146667,21.97,24.57,9.3,31.62,20.48,74.52,0.0,120.32,41.65,113.5,41.95,29.59,1.38,51.91,71.26,92.33,12.58,7.1,0.0,7.92,20.67,49.21,26.54,10.47,81.633,39.71,66.39,4.86,6.37,9.8,10.77,15.68,111.86,3.85,0.47,…,58.34,71.88,2.26,40.2,74.17,60.0,8.39,7.08,10.93,41.14,8.53,10.31,14.44,136.68,118.74,21.9,0.0,60.61,3.68,7.31,3.4,28.85,20.53,84.88,15.32,6.99,65.0,85.77,106.76,8.78,118.36,24.51,6.49,7.59,3.71,7.14,15.142
1992.0,47.193333,21.97,24.57,9.3,31.62,20.48,73.86,0.0,115.28,43.44,109.61,41.95,29.59,1.38,51.91,73.4,92.33,12.58,7.1,0.0,7.92,20.67,48.1725,26.54,11.45,81.633,39.71,66.89,4.86,6.37,9.8,10.77,15.68,111.48,3.85,0.47,…,58.34,71.88,2.26,40.2,74.17,60.11,8.39,7.08,10.93,41.83,9.2,10.31,14.44,140.75,118.74,21.9,0.0,57.46,3.68,7.72,3.4,28.85,20.53,85.99,15.32,6.99,65.0,85.77,107.42,8.78,118.57,24.51,6.49,7.59,3.71,7.14,15.324
1993.0,48.24,21.97,24.57,9.3,31.62,20.93,74.89,0.0,110.44,45.94,110.24,41.95,29.59,1.38,51.91,76.73,92.33,12.58,7.1,0.0,7.92,20.67,47.135,26.54,12.65,81.633,39.71,69.62,4.86,6.37,9.8,10.77,15.68,110.45,3.85,0.47,…,58.34,71.88,2.26,40.2,74.17,58.97,8.39,7.08,10.93,42.34,10.84,10.31,14.44,144.53,118.74,21.9,0.0,56.54,3.68,

In [17]:
# Reformat the dataframe with the filled values...
# Convert the DataFrame back to pandas!!!
wide_df = wide_pl_df.to_pandas() 

# Now we can reshape it with pandas functionality
complete_df = wide_df.melt(id_vars='Year', value_vars=wide_df.columns[1:], var_name='Country', value_name='RN/Midwife per 10k')

# Removing "_MD per 10k" from country names
complete_df['Country'] = complete_df['Country'].str.replace('_RN/Midwife per 10k', '')

complete_df['Year'] = complete_df['Year'].astype(int)

# Creating 'Country_Year' column
complete_df['Country_Year'] = complete_df['Country'].astype(str) + '_' + complete_df['Year'].astype(str)

complete_df = complete_df.loc[:, ['Country_Year', 'Country', 'Year', 'RN/Midwife per 10k']]
#complete_df['Country_Year'] = complete_df['Country_Year'].astype(str)


In [18]:
complete_df

,Country_Year,Country,Year,RN/Midwife per 10k
0,Albania_1990,Albania,1990,45.100000
1,Albania_1991,Albania,1991,46.146667
2,Albania_1992,Albania,1992,47.193333
3,Albania_1993,Albania,1993,48.240000
4,Albania_1994,Albania,1994,49.580000
...,...,...,...,...
6489,Zimbabwe_2019,Zimbabwe,2019,19.414500
6490,Zimbabwe_2020,Zimbabwe,2020,20.289000
6491,Zimbabwe_2021,Zimbabwe,2021,25.499500
6492,Zimbabwe_2022,Zimbabwe,2022,30.710000


In [19]:
complete_df.to_csv('../../Clean_Data/Clean_CSV_Files/RN-Midwife_per_10k.csv', index=False, encoding="utf-8")